# Diffusion Models vs GANs: Сравнение для финансовых временных рядов

В этом ноутбуке мы сравниваем два подхода к генеративному моделированию:

1. **GANs (Generative Adversarial Networks)** - из Главы 21
2. **Diffusion Models** - из этой главы

## Ключевые различия:

| Аспект | GANs | Diffusion |  
|--------|------|-----------|  
| Обучение | Adversarial (min-max game) | Простой MSE loss |
| Стабильность | Mode collapse, нестабильность | Стабильное обучение |
| Качество | Высокое, но вариативное | Высокое, консистентное |
| Скорость генерации | Быстрая (один forward pass) | Медленная (много итераций) |
| Вероятности | Implicit | Explicit (можно оценить likelihood) |
| Контроль | Сложный | Гибкий (conditional, guidance) |

In [ ]:
# Установка зависимостей
!pip install torch numpy pandas matplotlib seaborn scikit-learn tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Optional, Dict, List
from dataclasses import dataclass
from tqdm import tqdm
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import time
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Генерация тестовых данных

In [ ]:
def generate_financial_data(
    n_samples: int = 1000,
    seq_len: int = 50,
    seed: int = 42
) -> np.ndarray:
    """
    Генерация финансовых временных рядов (log returns).
    """
    np.random.seed(seed)
    
    data = []
    
    for i in range(n_samples):
        # Случайные параметры для разнообразия
        mu = np.random.uniform(-0.001, 0.001)
        sigma = np.random.uniform(0.01, 0.03)
        
        # GBM returns
        returns = np.random.normal(mu, sigma, seq_len)
        
        # Добавляем автокорреляцию (GARCH-like effect)
        vol = np.abs(returns)
        for t in range(1, seq_len):
            vol[t] = 0.1 * vol[t-1] + 0.9 * vol[t]
        returns = returns * (vol / vol.mean())
        
        data.append(returns)
    
    data = np.array(data)
    
    # Нормализация
    mean = data.mean()
    std = data.std()
    data = (data - mean) / std
    
    return data, mean, std

# Генерируем данные
data, data_mean, data_std = generate_financial_data(n_samples=2000, seq_len=50)
print(f"Data shape: {data.shape}")
print(f"Mean: {data.mean():.4f}, Std: {data.std():.4f}")

In [ ]:
# Визуализация реальных данных
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Примеры временных рядов
ax1 = axes[0]
for i in range(20):
    ax1.plot(data[i], alpha=0.5)
ax1.set_title('Sample Financial Time Series')
ax1.set_xlabel('Time')
ax1.set_ylabel('Normalized Return')

# Распределение
ax2 = axes[1]
ax2.hist(data.flatten(), bins=100, density=True, alpha=0.7)
ax2.set_title('Distribution of Returns')
ax2.set_xlabel('Return')
ax2.set_ylabel('Density')

plt.tight_layout()
plt.show()

In [ ]:
# Подготовка DataLoader
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, data: np.ndarray):
        self.data = torch.FloatTensor(data).unsqueeze(-1)  # [N, seq_len, 1]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Split
train_data = data[:1600]
test_data = data[1600:]

train_dataset = TimeSeriesDataset(train_data)
test_dataset = TimeSeriesDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Train: {len(train_dataset)}, Test: {len(test_dataset)}")

## 2. GAN Implementation (TimeGAN-style)

Реализуем упрощённую версию TimeGAN для временных рядов.

In [ ]:
class GANGenerator(nn.Module):
    """Generator для временных рядов."""
    
    def __init__(
        self,
        latent_dim: int = 32,
        hidden_dim: int = 128,
        seq_len: int = 50,
        output_dim: int = 1
    ):
        super().__init__()
        
        self.latent_dim = latent_dim
        self.seq_len = seq_len
        
        # LSTM-based generator
        self.lstm = nn.LSTM(
            input_size=latent_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0.1
        )
        
        self.output = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim // 2, output_dim),
            nn.Tanh()  # Output in [-1, 1]
        )
    
    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """
        Args:
            z: [batch, seq_len, latent_dim] - latent noise
        Returns:
            [batch, seq_len, output_dim]
        """
        h, _ = self.lstm(z)
        out = self.output(h)
        return out


class GANDiscriminator(nn.Module):
    """Discriminator для временных рядов."""
    
    def __init__(
        self,
        input_dim: int = 1,
        hidden_dim: int = 128
    ):
        super().__init__()
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=True
        )
        
        self.output = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: [batch, seq_len, input_dim]
        Returns:
            [batch, 1] - real/fake score
        """
        h, _ = self.lstm(x)
        # Global average pooling
        h = h.mean(dim=1)
        out = self.output(h)
        return out

In [ ]:
class TimeSeriesGAN:
    """GAN для генерации временных рядов."""
    
    def __init__(
        self,
        latent_dim: int = 32,
        hidden_dim: int = 128,
        seq_len: int = 50,
        device: torch.device = torch.device('cpu')
    ):
        self.latent_dim = latent_dim
        self.seq_len = seq_len
        self.device = device
        
        self.generator = GANGenerator(
            latent_dim=latent_dim,
            hidden_dim=hidden_dim,
            seq_len=seq_len
        ).to(device)
        
        self.discriminator = GANDiscriminator(
            input_dim=1,
            hidden_dim=hidden_dim
        ).to(device)
    
    def sample_latent(self, batch_size: int) -> torch.Tensor:
        """Sample latent noise."""
        return torch.randn(batch_size, self.seq_len, self.latent_dim, device=self.device)
    
    def generate(self, n_samples: int) -> torch.Tensor:
        """Generate synthetic samples."""
        self.generator.eval()
        with torch.no_grad():
            z = self.sample_latent(n_samples)
            samples = self.generator(z)
        return samples
    
    def train(
        self,
        loader: DataLoader,
        n_epochs: int = 50,
        lr: float = 0.0002
    ) -> Dict[str, List[float]]:
        """Train GAN."""
        
        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(0.5, 0.999))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
        
        criterion = nn.BCEWithLogitsLoss()
        
        history = {'g_loss': [], 'd_loss': []}
        
        for epoch in tqdm(range(n_epochs), desc="Training GAN"):
            g_losses = []
            d_losses = []
            
            for real_data in loader:
                real_data = real_data.to(self.device)
                batch_size = real_data.shape[0]
                
                # Labels
                real_labels = torch.ones(batch_size, 1, device=self.device)
                fake_labels = torch.zeros(batch_size, 1, device=self.device)
                
                # ---- Train Discriminator ----
                opt_d.zero_grad()
                
                # Real data
                real_pred = self.discriminator(real_data)
                d_loss_real = criterion(real_pred, real_labels)
                
                # Fake data
                z = self.sample_latent(batch_size)
                fake_data = self.generator(z).detach()
                fake_pred = self.discriminator(fake_data)
                d_loss_fake = criterion(fake_pred, fake_labels)
                
                d_loss = (d_loss_real + d_loss_fake) / 2
                d_loss.backward()
                opt_d.step()
                
                # ---- Train Generator ----
                opt_g.zero_grad()
                
                z = self.sample_latent(batch_size)
                fake_data = self.generator(z)
                fake_pred = self.discriminator(fake_data)
                g_loss = criterion(fake_pred, real_labels)  # Fool discriminator
                
                g_loss.backward()
                opt_g.step()
                
                g_losses.append(g_loss.item())
                d_losses.append(d_loss.item())
            
            history['g_loss'].append(np.mean(g_losses))
            history['d_loss'].append(np.mean(d_losses))
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}: G_loss={history['g_loss'][-1]:.4f}, D_loss={history['d_loss'][-1]:.4f}")
        
        return history

# Create and train GAN
gan = TimeSeriesGAN(latent_dim=32, hidden_dim=128, seq_len=50, device=device)
print(f"Generator params: {sum(p.numel() for p in gan.generator.parameters()):,}")
print(f"Discriminator params: {sum(p.numel() for p in gan.discriminator.parameters()):,}")

In [ ]:
# Train GAN
start_time = time.time()
gan_history = gan.train(train_loader, n_epochs=50, lr=0.0002)
gan_train_time = time.time() - start_time
print(f"\nGAN Training time: {gan_train_time:.1f}s")

In [ ]:
# Plot GAN training
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(gan_history['g_loss'], label='Generator Loss')
ax.plot(gan_history['d_loss'], label='Discriminator Loss')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('GAN Training Progress')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 3. Diffusion Model Implementation

In [ ]:
class SimpleDiffusion(nn.Module):
    """Простая Diffusion модель для временных рядов."""
    
    def __init__(
        self,
        seq_len: int = 50,
        hidden_dim: int = 128,
        num_layers: int = 4,
        diffusion_steps: int = 100
    ):
        super().__init__()
        
        self.seq_len = seq_len
        self.diffusion_steps = diffusion_steps
        
        # Time embedding
        self.time_embed = nn.Sequential(
            nn.Linear(1, hidden_dim),
            nn.SiLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Input projection
        self.input_proj = nn.Linear(1, hidden_dim)
        
        # Positional encoding
        self.pos_embed = nn.Parameter(torch.randn(1, seq_len, hidden_dim) * 0.02)
        
        # Transformer layers
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=4,
                dim_feedforward=hidden_dim * 4,
                dropout=0.1,
                batch_first=True
            )
            for _ in range(num_layers)
        ])
        
        # Output projection
        self.output_proj = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )
        
        # Noise schedule
        self._setup_schedule()
    
    def _setup_schedule(self):
        """Cosine schedule."""
        T = self.diffusion_steps
        s = 0.008
        
        steps = torch.arange(T + 1) / T
        alphas_bar = torch.cos((steps + s) / (1 + s) * np.pi / 2) ** 2
        alphas_bar = alphas_bar / alphas_bar[0]
        
        betas = 1 - alphas_bar[1:] / alphas_bar[:-1]
        betas = betas.clamp(max=0.999)
        
        alphas = 1 - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        
        self.register_buffer('betas', betas)
        self.register_buffer('alphas', alphas)
        self.register_buffer('alphas_cumprod', alphas_cumprod)
        self.register_buffer('sqrt_alphas_cumprod', torch.sqrt(alphas_cumprod))
        self.register_buffer('sqrt_one_minus_alphas_cumprod', torch.sqrt(1 - alphas_cumprod))
    
    def forward_diffusion(
        self,
        x0: torch.Tensor,
        t: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Forward diffusion."""
        noise = torch.randn_like(x0)
        
        sqrt_alpha = self.sqrt_alphas_cumprod[t].view(-1, 1, 1)
        sqrt_one_minus_alpha = self.sqrt_one_minus_alphas_cumprod[t].view(-1, 1, 1)
        
        xt = sqrt_alpha * x0 + sqrt_one_minus_alpha * noise
        
        return xt, noise
    
    def denoise(self, xt: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        """Predict noise."""
        batch_size = xt.shape[0]
        
        # Input projection
        h = self.input_proj(xt) + self.pos_embed
        
        # Time embedding
        t_emb = self.time_embed(t.float().unsqueeze(-1) / self.diffusion_steps)
        h = h + t_emb.unsqueeze(1)
        
        # Transformer layers
        for layer in self.layers:
            h = layer(h)
        
        # Output
        return self.output_proj(h)
    
    def compute_loss(self, x: torch.Tensor) -> torch.Tensor:
        """Compute training loss."""
        batch_size = x.shape[0]
        t = torch.randint(0, self.diffusion_steps, (batch_size,), device=x.device)
        
        xt, noise = self.forward_diffusion(x, t)
        noise_pred = self.denoise(xt, t)
        
        return F.mse_loss(noise_pred, noise)
    
    @torch.no_grad()
    def sample(self, n_samples: int, device: torch.device) -> torch.Tensor:
        """Generate samples via DDPM."""
        x = torch.randn(n_samples, self.seq_len, 1, device=device)
        
        for t in tqdm(reversed(range(self.diffusion_steps)), desc='Sampling', leave=False):
            t_tensor = torch.full((n_samples,), t, device=device)
            noise_pred = self.denoise(x, t_tensor)
            
            alpha = self.alphas[t]
            alpha_bar = self.alphas_cumprod[t]
            beta = self.betas[t]
            
            mean = (1 / torch.sqrt(alpha)) * (
                x - (beta / torch.sqrt(1 - alpha_bar)) * noise_pred
            )
            
            if t > 0:
                alpha_bar_prev = self.alphas_cumprod[t - 1]
                variance = beta * (1 - alpha_bar_prev) / (1 - alpha_bar)
                noise = torch.randn_like(x)
                x = mean + torch.sqrt(variance) * noise
            else:
                x = mean
        
        return x
    
    @torch.no_grad()
    def sample_ddim(self, n_samples: int, device: torch.device, num_steps: int = 20) -> torch.Tensor:
        """Fast sampling via DDIM."""
        x = torch.randn(n_samples, self.seq_len, 1, device=device)
        
        # Subset of timesteps
        step_size = self.diffusion_steps // num_steps
        timesteps = list(range(0, self.diffusion_steps, step_size))[::-1]
        
        for i, t in enumerate(tqdm(timesteps, desc='DDIM Sampling', leave=False)):
            t_tensor = torch.full((n_samples,), t, device=device)
            noise_pred = self.denoise(x, t_tensor)
            
            alpha_bar = self.alphas_cumprod[t]
            
            # Predicted x0
            x0_pred = (x - torch.sqrt(1 - alpha_bar) * noise_pred) / torch.sqrt(alpha_bar)
            
            if i < len(timesteps) - 1:
                t_prev = timesteps[i + 1]
                alpha_bar_prev = self.alphas_cumprod[t_prev]
                
                # DDIM update (deterministic)
                x = torch.sqrt(alpha_bar_prev) * x0_pred + \
                    torch.sqrt(1 - alpha_bar_prev) * noise_pred
            else:
                x = x0_pred
        
        return x

# Create diffusion model
diffusion = SimpleDiffusion(
    seq_len=50,
    hidden_dim=128,
    num_layers=4,
    diffusion_steps=100
).to(device)

print(f"Diffusion params: {sum(p.numel() for p in diffusion.parameters()):,}")

In [ ]:
# Train Diffusion
optimizer = torch.optim.AdamW(diffusion.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

n_epochs = 50
diffusion_losses = []

start_time = time.time()

for epoch in tqdm(range(n_epochs), desc="Training Diffusion"):
    diffusion.train()
    epoch_losses = []
    
    for batch in train_loader:
        batch = batch.to(device)
        
        optimizer.zero_grad()
        loss = diffusion.compute_loss(batch)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(diffusion.parameters(), 1.0)
        optimizer.step()
        
        epoch_losses.append(loss.item())
    
    scheduler.step()
    diffusion_losses.append(np.mean(epoch_losses))
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}: Loss={diffusion_losses[-1]:.4f}")

diffusion_train_time = time.time() - start_time
print(f"\nDiffusion Training time: {diffusion_train_time:.1f}s")

In [ ]:
# Plot Diffusion training
plt.figure(figsize=(10, 5))
plt.plot(diffusion_losses, label='Diffusion Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Diffusion Model Training Progress')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Генерация и сравнение сэмплов

In [ ]:
# Generate samples from both models
n_samples = 500

# GAN samples
print("Generating GAN samples...")
start_time = time.time()
gan_samples = gan.generate(n_samples).cpu().numpy().squeeze(-1)
gan_gen_time = time.time() - start_time

# Diffusion samples (DDPM)
print("Generating Diffusion samples (DDPM)...")
diffusion.eval()
start_time = time.time()
diffusion_samples = diffusion.sample(n_samples, device).cpu().numpy().squeeze(-1)
diffusion_gen_time = time.time() - start_time

# Diffusion samples (DDIM - fast)
print("Generating Diffusion samples (DDIM)...")
start_time = time.time()
diffusion_ddim_samples = diffusion.sample_ddim(n_samples, device, num_steps=20).cpu().numpy().squeeze(-1)
diffusion_ddim_gen_time = time.time() - start_time

print(f"\nGAN generation time: {gan_gen_time:.2f}s ({n_samples/gan_gen_time:.1f} samples/s)")
print(f"DDPM generation time: {diffusion_gen_time:.2f}s ({n_samples/diffusion_gen_time:.1f} samples/s)")
print(f"DDIM generation time: {diffusion_ddim_gen_time:.2f}s ({n_samples/diffusion_ddim_gen_time:.1f} samples/s)")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Row 1: Sample time series
for i, (ax, samples, name) in enumerate(zip(axes[0], 
                                             [test_data[:20], gan_samples[:20], diffusion_samples[:20]],
                                             ['Real', 'GAN', 'Diffusion'])):
    for s in samples:
        ax.plot(s, alpha=0.5)
    ax.set_title(f'{name} Samples')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')

# Row 2: Distributions
for i, (ax, samples, name) in enumerate(zip(axes[1],
                                             [test_data, gan_samples, diffusion_samples],
                                             ['Real', 'GAN', 'Diffusion'])):
    ax.hist(samples.flatten(), bins=100, density=True, alpha=0.7)
    ax.set_title(f'{name} Distribution')
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')

plt.tight_layout()
plt.show()

## 5. Количественное сравнение

In [ ]:
def compute_metrics(real: np.ndarray, synthetic: np.ndarray) -> Dict[str, float]:
    """Compute quality metrics for synthetic data."""
    
    metrics = {}
    
    # Statistical metrics
    real_flat = real.flatten()
    synth_flat = synthetic.flatten()
    
    # Mean absolute difference
    metrics['mean_diff'] = abs(np.mean(real_flat) - np.mean(synth_flat))
    metrics['std_diff'] = abs(np.std(real_flat) - np.std(synth_flat))
    
    # Skewness and Kurtosis
    from scipy.stats import skew, kurtosis
    metrics['skew_diff'] = abs(skew(real_flat) - skew(synth_flat))
    metrics['kurtosis_diff'] = abs(kurtosis(real_flat) - kurtosis(synth_flat))
    
    # Autocorrelation
    def acf(x, lag):
        return np.corrcoef(x[:-lag], x[lag:])[0, 1]
    
    real_acf = [acf(real_flat, lag) for lag in [1, 5, 10]]
    synth_acf = [acf(synth_flat, lag) for lag in [1, 5, 10]]
    metrics['acf_mae'] = np.mean(np.abs(np.array(real_acf) - np.array(synth_acf)))
    
    # Discriminative score
    n_real = min(len(real), 500)
    n_synth = min(len(synthetic), 500)
    
    X = np.vstack([real[:n_real].reshape(n_real, -1), 
                   synthetic[:n_synth].reshape(n_synth, -1)])
    y = np.array([0] * n_real + [1] * n_synth)
    
    clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
    scores = cross_val_score(clf, X, y, cv=3)
    metrics['discriminative_score'] = scores.mean()
    
    return metrics

In [ ]:
# Compute metrics for both models
print("Computing metrics...")

gan_metrics = compute_metrics(test_data, gan_samples)
diffusion_metrics = compute_metrics(test_data, diffusion_samples)
ddim_metrics = compute_metrics(test_data, diffusion_ddim_samples)

# Display results
print("\n" + "=" * 70)
print("COMPARISON: GAN vs Diffusion Models")
print("=" * 70)

print(f"\n{'Metric':<25} {'GAN':<15} {'DDPM':<15} {'DDIM':<15}")
print("-" * 70)

for metric in gan_metrics.keys():
    gan_val = gan_metrics[metric]
    diff_val = diffusion_metrics[metric]
    ddim_val = ddim_metrics[metric]
    
    # Highlight better value
    if metric == 'discriminative_score':
        # Closer to 0.5 is better
        gan_dist = abs(gan_val - 0.5)
        diff_dist = abs(diff_val - 0.5)
        ddim_dist = abs(ddim_val - 0.5)
        best = min(gan_dist, diff_dist, ddim_dist)
        gan_mark = '*' if gan_dist == best else ' '
        diff_mark = '*' if diff_dist == best else ' '
        ddim_mark = '*' if ddim_dist == best else ' '
    else:
        # Lower is better
        best = min(gan_val, diff_val, ddim_val)
        gan_mark = '*' if gan_val == best else ' '
        diff_mark = '*' if diff_val == best else ' '
        ddim_mark = '*' if ddim_val == best else ' '
    
    print(f"{metric:<25} {gan_val:>14.4f}{gan_mark} {diff_val:>14.4f}{diff_mark} {ddim_val:>14.4f}{ddim_mark}")

print("\n* = best value")
print("\nNote: discriminative_score closer to 0.5 = harder to distinguish from real")

In [ ]:
# Speed comparison
print("\n" + "=" * 50)
print("SPEED COMPARISON")
print("=" * 50)

print(f"\n{'Metric':<25} {'GAN':<15} {'DDPM':<15} {'DDIM':<15}")
print("-" * 70)
print(f"{'Training time (s)':<25} {gan_train_time:<15.1f} {diffusion_train_time:<15.1f} {'-':^15}")
print(f"{'Generation time (s)':<25} {gan_gen_time:<15.2f} {diffusion_gen_time:<15.2f} {diffusion_ddim_gen_time:<15.2f}")
print(f"{'Samples per second':<25} {n_samples/gan_gen_time:<15.1f} {n_samples/diffusion_gen_time:<15.1f} {n_samples/diffusion_ddim_gen_time:<15.1f}")
print(f"{'Speedup vs DDPM':<25} {diffusion_gen_time/gan_gen_time:<15.1f}x {1.0:<15.1f}x {diffusion_gen_time/diffusion_ddim_gen_time:<15.1f}x")

## 6. t-SNE Visualization

In [ ]:
# t-SNE embedding
n_vis = 300

# Prepare data
real_vis = test_data[:n_vis].reshape(n_vis, -1)
gan_vis = gan_samples[:n_vis].reshape(n_vis, -1)
diff_vis = diffusion_samples[:n_vis].reshape(n_vis, -1)

combined = np.vstack([real_vis, gan_vis, diff_vis])
labels = np.array([0] * n_vis + [1] * n_vis + [2] * n_vis)

print("Computing t-SNE...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
embedded = tsne.fit_transform(combined)

# Plot
plt.figure(figsize=(10, 8))
colors = ['blue', 'red', 'green']
names = ['Real', 'GAN', 'Diffusion']

for i, (color, name) in enumerate(zip(colors, names)):
    mask = labels == i
    plt.scatter(
        embedded[mask, 0], embedded[mask, 1],
        c=color, label=name, alpha=0.5, s=20
    )

plt.title('t-SNE: Real vs GAN vs Diffusion')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Diversity Analysis

In [ ]:
def compute_diversity(samples: np.ndarray) -> Dict[str, float]:
    """Compute diversity metrics."""
    n = len(samples)
    flat = samples.reshape(n, -1)
    
    # Pairwise distances
    from sklearn.metrics import pairwise_distances
    dists = pairwise_distances(flat[:200], metric='euclidean')
    
    # Mean pairwise distance (higher = more diverse)
    upper_tri = dists[np.triu_indices(len(dists), k=1)]
    
    return {
        'mean_distance': np.mean(upper_tri),
        'std_distance': np.std(upper_tri),
        'min_distance': np.min(upper_tri),
        'max_distance': np.max(upper_tri)
    }

real_diversity = compute_diversity(test_data)
gan_diversity = compute_diversity(gan_samples)
diffusion_diversity = compute_diversity(diffusion_samples)

print("\n" + "=" * 60)
print("DIVERSITY ANALYSIS")
print("=" * 60)
print(f"\n{'Metric':<20} {'Real':<15} {'GAN':<15} {'Diffusion':<15}")
print("-" * 60)

for metric in real_diversity.keys():
    print(f"{metric:<20} {real_diversity[metric]:<15.4f} {gan_diversity[metric]:<15.4f} {diffusion_diversity[metric]:<15.4f}")

print("\nNote: Higher mean_distance = more diverse samples")

## 8. Summary and Conclusions

In [ ]:
# Summary visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Training stability
ax1 = axes[0, 0]
ax1.plot(gan_history['g_loss'], label='GAN Generator', alpha=0.7)
ax1.plot(gan_history['d_loss'], label='GAN Discriminator', alpha=0.7)
ax1.plot(diffusion_losses, label='Diffusion', alpha=0.7)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Stability')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Distribution comparison
ax2 = axes[0, 1]
ax2.hist(test_data.flatten(), bins=100, alpha=0.5, label='Real', density=True)
ax2.hist(gan_samples.flatten(), bins=100, alpha=0.5, label='GAN', density=True)
ax2.hist(diffusion_samples.flatten(), bins=100, alpha=0.5, label='Diffusion', density=True)
ax2.set_xlabel('Value')
ax2.set_ylabel('Density')
ax2.set_title('Distribution Comparison')
ax2.legend()
ax2.set_xlim(-4, 4)

# 3. Speed comparison
ax3 = axes[1, 0]
models = ['GAN', 'DDPM', 'DDIM']
gen_times = [gan_gen_time, diffusion_gen_time, diffusion_ddim_gen_time]
bars = ax3.bar(models, gen_times, color=['red', 'blue', 'green'])
ax3.set_ylabel('Generation Time (s)')
ax3.set_title(f'Generation Speed ({n_samples} samples)')
for bar, t in zip(bars, gen_times):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{t:.2f}s', ha='center', va='bottom')

# 4. Quality metrics
ax4 = axes[1, 1]
metrics_names = ['mean_diff', 'std_diff', 'skew_diff', 'acf_mae']
x = np.arange(len(metrics_names))
width = 0.35

gan_vals = [gan_metrics[m] for m in metrics_names]
diff_vals = [diffusion_metrics[m] for m in metrics_names]

bars1 = ax4.bar(x - width/2, gan_vals, width, label='GAN', color='red', alpha=0.7)
bars2 = ax4.bar(x + width/2, diff_vals, width, label='Diffusion', color='blue', alpha=0.7)

ax4.set_ylabel('Error (lower is better)')
ax4.set_title('Quality Metrics')
ax4.set_xticks(x)
ax4.set_xticklabels(metrics_names, rotation=45)
ax4.legend()

plt.tight_layout()
plt.show()

## 9. Выводы

### GAN (Generative Adversarial Networks)

**Преимущества:**
- Очень быстрая генерация (один forward pass)
- Хорошо изучены, много архитектур
- Могут генерировать очень реалистичные данные

**Недостатки:**
- Mode collapse (потеря разнообразия)
- Нестабильное обучение (adversarial training)
- Трудно оценить качество без дополнительных метрик
- Сложная настройка гиперпараметров

### Diffusion Models

**Преимущества:**
- Стабильное обучение (простой MSE loss)
- Высокое качество и разнообразие
- Можно оценить likelihood
- Гибкое кондиционирование и guidance
- Лучше для imputation и conditional generation

**Недостатки:**
- Медленная генерация (много итераций)
- Требует больше памяти при обучении
- Более новая область, меньше готовых решений

### Рекомендации по выбору

| Задача | Рекомендуемый подход |
|--------|---------------------|
| Real-time генерация | GAN или DDIM |
| Высокое качество | Diffusion (DDPM) |
| Imputation/Forecasting | CSDI или TimeGrad |
| Data augmentation | Оба подхода работают |
| Стабильность обучения | Diffusion |
| Низкие вычислительные ресурсы | GAN |

In [ ]:
# Save models and results
torch.save({
    'generator_state_dict': gan.generator.state_dict(),
    'discriminator_state_dict': gan.discriminator.state_dict(),
    'history': gan_history,
    'metrics': gan_metrics
}, 'gan_model.pt')

torch.save({
    'model_state_dict': diffusion.state_dict(),
    'losses': diffusion_losses,
    'metrics': diffusion_metrics
}, 'diffusion_model.pt')

# Save comparison results
comparison_results = {
    'gan_metrics': gan_metrics,
    'diffusion_metrics': diffusion_metrics,
    'ddim_metrics': ddim_metrics,
    'gan_train_time': gan_train_time,
    'diffusion_train_time': diffusion_train_time,
    'gan_gen_time': gan_gen_time,
    'diffusion_gen_time': diffusion_gen_time,
    'ddim_gen_time': diffusion_ddim_gen_time
}

import json
with open('comparison_results.json', 'w') as f:
    json.dump(comparison_results, f, indent=2)

print("Models and results saved!")